## Making MDPs in Python

For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

You can start this assignment during/after reading Grokking Ch-2.

NOTE: all page numbers referred to in this ipynb notebook refer to Grokking textbook

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [1]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

Here we a dictionary bw_mdp which to get the transitions possible we can do bw_mdp\[current state\]\[action to take\]. this will return a list of the possibilities.

The format of each element within the list is   
(probability of that transition , next state , reward , 'is the next state a terminal state?')

Obviously make sure that the sum of probability of all possible transitions is 1

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Slippery Walk

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

And similiarly vice versa for Left(just replace left with right and right with left in previous list)

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [2]:
swf_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)],
    },

    1 : {
        "Right" : [
            (1/2, 2, 0, False),
            (1/3, 1, 0, False),
            (1/6, 0, 0, True),
        ],
        "Left" : [
            (1/2, 0, 0, True),
            (1/3, 1, 0, False),
            (1/6, 2, 0, False),
        ]
    }, 
    
    2 : {
        "Right" : [
            (1/2, 3, 0, False),
            (1/3, 2, 0, False),
            (1/6, 1, 0, False),
        ],
        "Left" : [
            (1/2, 1, 0, False),
            (1/3, 2, 0, False),
            (1/6, 3, 0, False),
        ]
    },

    3 : {
        "Right" : [
            (1/2, 4, 0, False),
            (1/3, 3, 0, False),
            (1/6, 2, 0, False),
        ],
        "Left" : [
            (1/2, 2, 0, False),
            (1/3, 3, 0, False),
            (1/6, 4, 0, False),
        ]
    },

    4 : {
        "Right" : [
            (1/2, 5, 0, False),
            (1/3, 4, 0, False),
            (1/6, 3, 0, False),
        ],
        "Left" : [
            (1/2, 3, 0, False),
            (1/3, 4, 0, False),
            (1/6, 5, 0, False),
        ]
    },

    5 : {
        "Right" : [
            (1/2, 6, 1, True),
            (1/3, 5, 0, False),
            (1/6, 4, 0, False),
        ],
        "Left" : [
            (1/2, 4, 0, False),
            (1/3, 5, 0, False),
            (1/6, 6, 1, True),
        ]
    },

    6 : {
        "Right" : [(1, 6, 0, True)],
        "Left" : [(1, 6, 0, True)],
    },
    
}

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [3]:
fl_mdp = {
    # to be added
}

In [ ]:
for state in range(0, 16):

    transitions = {}
    
    for action in ["Up", "Down", "Right", "Left"]:
        
        if state in {5, 7, 11, 12, 15}:
            transitions[action] = [(1, state, 0, True)]
        else:
            transitions[action] = []
            if action == "Up":
                nextstates = [state - 4 if state // 4 > 0 else state,
                              state if state % 4 == 3 else state + 1,
                              state if state % 4 == 0 else state - 1]
            elif action == "Down":
                nextstates = [state + 4 if state // 4 < 3 else state,
                              state if state % 4 == 3 else state + 1,
                              state if state % 4 == 0 else state - 1]
            elif action == "Right":
                nextstates = [state if state % 4 == 3 else state + 1,
                              state - 4 if state // 4 > 0 else state,
                              state + 4 if state // 4 < 3 else state]
            else:
                nextstates = [state if state % 4 == 0 else state - 1,
                              state - 4 if state // 4 > 0 else state,
                              state + 4 if state // 4 < 3 else state]
            for next in nextstates:
                if next in {5, 7, 11, 12}:
                    transitions[action].append((1/3, next, 0, True))
                elif next == 15:
                    transitions[action].append((1/3, next, 1, True))
                else:
                    transitions[action].append((1/3, next, 0, False))

        continue

    fl_mdp[state] = transitions

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.
OpenAI Gym basically contains implmentations of common RL MDPs so that we can directly train our agents on test environments. 

In [ ]:
import gym
P = gym.make('FrozenLake-v1').env.P

ModuleNotFoundError: No module named 'gymnasium'

Since the imported MDP is also just a dictionary, we can just print it.

In [ ]:
# using the pretty print module

import pprint
pprint.pprint(P)

NameError: name 'P' is not defined

/bin/bash: line 1: pip3: command not found


## Environment 3 - Key Door Game

Now we will implement a slightly different version of frozen lake without the stochasticity but with the added constraint of a key.

So now we have a 4x4 grid in which some cells are holes(and so terminal) and we have a goal state(keep it as 15) but this time someone locked the goal state with a door :(. Without the key we CANNOT ENTER the goal state.

Luckily for us, that guy was very clumsy and dropped the key in tile 3!. So now make a MDP to represent the process of us escaping this grid world.

Remember that the state in a MDP isnt just the position/location we are in. it should represent our whole game situation. (For example I can either be at tile 14 with the key or at tile 14 without key and these two situations are different and therfore must be represented as **different** states!)

To summarise
- state state is 0
- door/goal state is 15
- holes at 5, 7, 11, 12
- key at 3
- The key is picked up automatically upon entering tile 3
- can only enter tile 15 when key is obtained(otherwise its like a wall)
- holes & goal are terminal
- reward of 1 when reaching tile 15 wiht the key
- Attempting to enter tile 15 when we DONT have the key should be treated like hitting a wall and bouncing back to the original state where we took that action

In [ ]:
# You can implements any helper function or other variables to help you implement it

gw_map = {
    #your mdp should go here
}

for state in range(0, 16):
    transitions = {}
    for action in ["Up", "Down", "Right", "Left"]:
        if state in {5, 7, 11, 12, 15}:
            transitions[action] = [(1, (state, True), 0, True)]
        else:
            if action == "Up":
                next = state - 4 if state // 4 > 0 else state                            
            elif action == "Down":
                next = state + 4 if state // 4 < 3 else state
            elif action == "Right":
                next = state if state % 4 == 3 else state + 1
            else:
                next = state if state % 4 == 0 else state - 1
            if next in {5, 7, 11, 12}:
                transitions[action] = [(1, (next, True), 0, True)]
            elif next == 15:
                transitions[action] = [(1, (next, True), 1, True)]
            else:
                transitions[action] = [(1, (next, True), 0, False)]
    gw_map[(state, True)] = transitions
    
for state in range(0, 15):
    if state == 3:
        continue
    transitions = {}
    for action in ["Up", "Down", "Right", "Left"]:
        if state in {5, 7, 11, 12}:
            transitions[action] = [(1, (state, False), 0, True)]
        else:
            transitions[action] = []
            if action == "Up":
                next = state - 4 if state // 4 > 0 else state
            elif action == "Down":
                next = state + 4 if state // 4 < 3 else state
            elif action == "Right":
                next = state if state % 4 == 3 else state + 1
            else:
                next = state if state % 4 == 0 else state - 1
            if next in {5, 7, 11, 12}:
                transitions[action] = [(1/3, (next, False), 0, True)]
            elif next == 15:
                transitions[action] = [(1/3, (state, False), 0, False)]
            elif next == 3:
                transitions[action] = [(1/3, (state, True), 0, False)]
            else:
                transitions[action] = [(1/3, (next, False), 0, False)]
    gw_map[(state, False)] = transitions

The exact way you represent each state and each action is up to you! But... do atleast write down in a comment or something how you represent the states and action in the MDP. So i don't have to spend time deciphering.

Note: For additional challenges
- Make this grid-world also stochastic like Frozen Lake
- Make a function that allows you to input in the locations of the holes, the door(goal state) and the key and using that it will generate and return the MDP

In [ ]:
gw_map_stochastic = {
}

for state in range(0, 16):
    transitions = {}
    for action in ["Up", "Down", "Right", "Left"]:
        if state in {5, 7, 11, 12, 15}:
            transitions[action] = [(1, (state, True), 0, True)]
        else:
            transitions[action] = []
            if action == "Up":
                nextstates = [state - 4 if state // 4 > 0 else state,
                            state if state % 4 == 3 else state + 1,
                            state if state % 4 == 0 else state - 1]
            elif action == "Down":
                nextstates = [state + 4 if state // 4 < 3 else state,
                            state if state % 4 == 3 else state + 1,
                            state if state % 4 == 0 else state - 1]
            elif action == "Right":
                nextstates = [state if state % 4 == 3 else state + 1,
                            state - 4 if state // 4 > 0 else state,
                            state + 4 if state // 4 < 3 else state]
            else:
                nextstates = [state if state % 4 == 0 else state - 1,
                            state - 4 if state // 4 > 0 else state,
                            state + 4 if state // 4 < 3 else state]
            for next in nextstates:
                if next in {5, 7, 11, 12}:
                    transitions[action].append((1/3, (next, True), 0, True))
                elif next == 15:
                    transitions[action].append((1/3, (next, True), 1, True))
                else:
                    transitions[action].append((1/3, (next, True), 0, False))
    gw_map_stochastic[(state, True)] = transitions
for state in range(0, 15):
    if state == 3:
        continue
    transitions = {}
    for action in ["Up", "Down", "Right", "Left"]:
        if state in {5, 7, 11, 12}:
            transitions[action] = [(1, (state, False), 0, True)]
        else:
            transitions[action] = []
            if action == "Up":
                nextstates = [state - 4 if state // 4 > 0 else state,
                              state if state % 4 == 3 else state + 1,
                              state if state % 4 == 0 else state - 1]
            elif action == "Down":
                nextstates = [state + 4 if state // 4 < 3 else state,
                              state if state % 4 == 3 else state + 1,
                              state if state % 4 == 0 else state - 1]
            elif action == "Right":
                nextstates = [state if state % 4 == 3 else state + 1,
                              state - 4 if state // 4 > 0 else state,
                              state + 4 if state // 4 < 3 else state]
            else:
                nextstates = [state if state % 4 == 0 else state - 1,
                              state - 4 if state // 4 > 0 else state,
                              state + 4 if state // 4 < 3 else state]
            for next in nextstates:
                if next in {5, 7, 11, 12}:
                    transitions[action].append((1/3, (next, False), 0, True))
                elif next == 15:
                    transitions[action].append((1/3, (state, False), 0, False))
                elif next == 3:
                    transitions[action].append((1/3, (state, True), 0, False))
                else:
                    transitions[action].append((1/3, (next, False), 0, False))
    gw_map_stochastic[(state, False)] = transitions

In [ ]:
def f(input_holes, input_door, input_key):
    holes = input_holes
    door = input_door
    key = input_key
    gw_map_stochastic_withinput = {
    }
    for state in range(0, 16):
        transitions = {}
        for action in ["Up", "Down", "Right", "Left"]:
            if state in holes or state == door:
                transitions[action] = [(1, (state, True), 0, True)]
            else:
                transitions[action] = []
                if action == "Up":
                    nextstates = [state - 4 if state // 4 > 0 else state,
                                 state if state % 4 == 3 else state + 1,
                                 state if state % 4 == 0 else state - 1]
                elif action == "Down":
                    nextstates = [state + 4 if state // 4 < 3 else state,
                                state if state % 4 == 3 else state + 1,
                                state if state % 4 == 0 else state - 1]
                elif action == "Right":
                    nextstates = [state if state % 4 == 3 else state + 1,
                                state - 4 if state // 4 > 0 else state,
                                state + 4 if state // 4 < 3 else state]
                elif action == "Right":
                    nextstates = [state if state % 4 == 3 else state + 1,
                                state - 4 if state // 4 > 0 else state,
                                state + 4 if state // 4 < 3 else state]
                else:
                    nextstates = [state if state % 4 == 0 else state - 1,
                                state - 4 if state // 4 > 0 else state,
                                state + 4 if state // 4 < 3 else state]
                for next in nextstates:
                    if next in holes:
                        transitions[action].append((1/3, (next, True), 0, True))
                    elif next == door:
                        transitions[action].append((1/3, (next, True), 1, True))
                    else:
                        transitions[action].append((1/3, (next, True), 0, False))
        gw_map_stochastic_withinput[(state, True)] = transitions
    for state in range(0, 16):
        if state == key or state == door:
            continue
        transitions = {}
        for action in ["Up", "Down", "Right", "Left"]:
            if state in holes:
                transitions[action] = [(1, (state, False), 0, True)]
            else:
                transitions[action] = []
                if action == "Up":
                    nextstates = [state - 4 if state // 4 > 0 else state,
                                  state if state % 4 == 3 else state + 1,
                                  state if state % 4 == 0 else state - 1]
                elif action == "Down":
                    nextstates = [state + 4 if state // 4 < 3 else state,
                                  state if state % 4 == 3 else state + 1,
                                  state if state % 4 == 0 else state - 1]
                elif action == "Right":
                    nextstates = [state if state % 4 == 3 else state + 1,
                                  state - 4 if state // 4 > 0 else state,
                                  state + 4 if state // 4 < 3 else state]
                else:
                    nextstates = [state if state % 4 == 0 else state - 1,
                                  state - 4 if state // 4 > 0 else state,
                                  state + 4 if state // 4 < 3 else state]
                for next in nextstates:
                    if next in holes:
                        transitions[action].append((1/3, (next, False), 0, True))
                    elif next == door:
                        transitions[action].append((1/3, (state, False), 0, False))
                    elif next == key:
                        transitions[action].append((1/3, (state, True), 0, False))
                    else:
                        transitions[action].append((1/3, (next, False), 0, False))
        gw_map_stochastic_withinput[(state, False)] = transitions
    return gw_map_stochastic_withinput

## Environment 4 - Lights Out

**Lights out** is a deterministic puzzle game which consists of a 5x5 grid of lights which can either be on or off.

When the game begins a random set of these lights will be switched on. Pressing any light will toggle it (toggle means if the light was on then it will become off and if it was off then it will become on) and it will also toggle all of its neighbours.

![image of the game](./images/LightsOutIllustration.png)

Note: so any non-edge and non-corner cell action will toggle 5 lights, an edge cell action would toggle 4 lights and a corner cell action only toggles 3 lights.


The goal of the game is to find a sequence of moves to switch off all the lights. Now i'm not gonna make you solve this instead we are going to represent this as a deterministic MDP, where 
- the states should represent the game grid layout 
- the actions are the cells you wish to toggle
- Give a reward of 1 when solved 
- the ONLY terminal state is the fully solved state

Since 5x5 is pretty big we will restrict ourselves to a **4x4** grid and represent it as an MDP instead

In [ ]:
lo_map = { 
}
for s in range(0, 2**16):
    t = {}
    for a in range(0, 16):
        if s == 0:
            t[a] = [(1, s, 0, True)]
        else:
            nx = s^(2**a + (0 if a % 4 == 0 else 2**(a - 1)) + (0 if a % 4 == 3 else 2**(a + 1)) + (0 if a // 4 == 0 else 2 ** (a - 4)) + (0 if a // 4 == 3 else 2**(a + 4)))
            if nx == 0:
                t[a] = [(1, nx, 1, True)]
            else:
                t[a] = [(1, nx, 0, False)]
    lo_map[s] = t

In [ ]:
lo_map

{0: {0: [(1, 0, 0, True)],
  1: [(1, 0, 0, True)],
  2: [(1, 0, 0, True)],
  3: [(1, 0, 0, True)],
  4: [(1, 0, 0, True)],
  5: [(1, 0, 0, True)],
  6: [(1, 0, 0, True)],
  7: [(1, 0, 0, True)],
  8: [(1, 0, 0, True)],
  9: [(1, 0, 0, True)],
  10: [(1, 0, 0, True)],
  11: [(1, 0, 0, True)],
  12: [(1, 0, 0, True)],
  13: [(1, 0, 0, True)],
  14: [(1, 0, 0, True)],
  15: [(1, 0, 0, True)]},
 1: {0: [(1, 18, 0, False)],
  1: [(1, 38, 0, False)],
  2: [(1, 79, 0, False)],
  3: [(1, 141, 0, False)],
  4: [(1, 304, 0, False)],
  5: [(1, 627, 0, False)],
  6: [(1, 1253, 0, False)],
  7: [(1, 2249, 0, False)],
  8: [(1, 4881, 0, False)],
  9: [(1, 10017, 0, False)],
  10: [(1, 20033, 0, False)],
  11: [(1, 35969, 0, False)],
  12: [(1, 12545, 0, False)],
  13: [(1, 29185, 0, False)],
  14: [(1, 58369, 0, False)],
  15: [(1, 51201, 0, False)]},
 2: {0: [(1, 17, 0, False)],
  1: [(1, 37, 0, False)],
  2: [(1, 76, 0, False)],
  3: [(1, 142, 0, False)],
  4: [(1, 307, 0, False)],
  5: [(1, 624, 

The exact way you represent each state and each action is up to you! But... do atleast write down in a comment or something how you represent the states and action in the MDP. So i don't have to spend time deciphering.

Note: if you want you can also try to represent the 5x5 one but this is optional